# **Intro**
preprocessing steps taken from here: 
https://colab.research.google.com/github/KrishnaswamyLab/SingleCellWorkshop/blob/master/exercises/Preprocessing/notebooks/00_Advanced_Loading_and_preprocessing_scRNAseq_data.ipynb#scrollTo=RUZx-nc_1Mdr

cell-type clustering inspiration: https://colab.research.google.com/github/KrishnaswamyLab/SingleCellWorkshop/blob/master/exercises/Deep_Learning/notebooks/00_Cell_type_classification_with_neural_networks.ipynb

method of simulation inspired by BayesPrism: https://www.biorxiv.org/content/10.1101/2020.01.07.897900v3.full#F1

Data taken from this paper: https://www.nature.com/articles/s41587-020-0465-8#Sec2

Gene Omnibus link: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE132044

Data was downloaded from the Single Cell Portal: https://singlecell.broadinstitute.org/single_cell/study/SCP424/single-cell-comparison-pbmc-data


# **Github / VM setup**

To integrate Colab with github, we need to do some setup first.
If you want this to integrate with your own github repo, you will need to create a `secrets` folder that contain your public and private key, and register this to access you github repo. Check `setup.sh` for more details.

In [1]:
import os, sys

NB_ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood'
sys.path.append('/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood')

In [2]:
# prelude: set up git, etc.
%cd {NB_ROOT_PATH}
!( source setup.sh )

/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood
Ensuring ssh keys exist...
Copied key(s) to /root/.ssh/
total 12
-rw-r--r-- 1 root root  43 Sep 22 20:33 config
-rw------- 1 root root 432 Sep 22 20:33 id_rsa
-rw-r--r-- 1 root root 113 Sep 22 20:33 id_rsa.pub


In [3]:
%%bash
# do your git operations here

git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   experiments/rotated_example_ssDIVA_mnist.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	bayesprism/
	experiments/pbmc1a_ssDIVA.ipynb
	method_comparison/
	results/
	sc_preprocessing/

no changes added to commit (use "git add" and/or "git commit -a")


# **Imports**

In [4]:
!pip install scprep tasklogger

     |████████████████████████████████| 104 kB 13.5 MB/s 


In [5]:
# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scprep
import scipy as sp
from scipy.sparse import coo_matrix

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE


# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import tasklogger
import pickle
import gzip
from pathlib import Path


# Parameters

In [6]:
# parameters
GSE_data_path = "/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood/data/single_cell_data/GSE132044/"
aug_data_path = "/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood/data/single_cell_data/augmented_pbmc_data/"
cybersort_path = "/content/drive/MyDrive/Colab Notebooks/checkouts/sc_bulk_ood/data/single_cell_data/cybersort_pbmc/"

smart_seq2_param = pd.DataFrame({"Method":'Smart-seq2', 
                    "Experiment":'pbmc1', 
                    "min_num_cells":[-1], 
                    "num_cells":[253],
                    "file_id":'pbmc_rep1_sm2'})

pbmc1_10x_param = pd.DataFrame({"Method":'10x Chromium V2 A', 
                    "Experiment":'pbmc1', 
                    "min_num_cells":[-1], 
                    "num_cells":[3222],
                    "file_id":'pbmc_rep1_10xV2a'})

pbmc2_10x_param = pd.DataFrame({"Method":'10x Chromium V2', 
                    "Experiment":'pbmc2', 
                    "min_num_cells":[-1], 
                    "num_cells":[3362],
                    "file_id":'pbmc_rep2_10xV2'})

pbmc1_10x_sm2_cells_param = pd.DataFrame({"Method":'10x Chromium V2 A', 
                    "Experiment":'pbmc1', 
                    "min_num_cells":[-1], 
                    "num_cells":[3222],
                    "file_id":'pbmc_rep1_10xV2a_sm2_cells'})

#####################
### set the study ###
#####################
curr_study = pbmc1_10x_sm2_cells_param

# set the parameters
min_num_cells = pd.to_numeric(curr_study["min_num_cells"][0])
method_keep = curr_study["Method"].tolist()
experiment_keep = curr_study["Experiment"].tolist()
num_cells_expected = pd.to_numeric(curr_study["num_cells"][0])
out_file_id = curr_study["file_id"][0]

if method_keep[0] == "Smart-seq2":
  # raw files
  cell_file = os.path.join(GSE_data_path, "GSE132044_cells_read_new.txt")
  count_file = os.path.join(GSE_data_path, "GSE132044_counts_read.txt.gz")
  gene_file = os.path.join(GSE_data_path, "GSE132044_genes_read.txt")

else:
  # raw files
  cell_file = os.path.join(GSE_data_path, "GSE132044_cells_umi_new.txt")
  count_file = os.path.join(GSE_data_path, "GSE132044_counts_umi.txt.gz")
  gene_file = os.path.join(GSE_data_path, "GSE132044_genes_umi.txt")

meta_file = os.path.join(GSE_data_path, "GSE132044_meta_counts_new.txt")
meta_tsne_file = os.path.join(GSE_data_path, "GSE132044_meta.txt")






# Load + Process Data

## Read data

In [7]:
cell_info = pd.read_table(cell_file, names=["cell_names"], header=None)
cell_info["idx"] = range(1,cell_info.shape[0]+1) # we need the index for later -- 1 indexed!

count_ptr = gzip.open(count_file, "r")
count_matr = pd.read_table(count_ptr, skiprows=2, names=["gene_idx", "cell_idx", "expr"], sep=" ")

gene_info = pd.read_table(gene_file, names=["gene_ids"], header=None)

meta_info = pd.read_table(meta_file)

# merge metadata with cell_type labels 
meta_tsne = pd.read_table(meta_tsne_file, skiprows=[1])
meta_tsne.rename(columns = {'NAME':'Name'}, inplace = True)
meta_info = meta_info.merge(meta_tsne[["Name", "CellType"]], on="Name")

print(cell_info.head())
print(cell_info.shape)

print(count_matr.head())
print(count_matr.shape)

print(gene_info.head())
print(gene_info.shape)

print(meta_info.head())
print(meta_info.shape)



               cell_names  idx
0  pbmc1_Celseq2_1_ACAGAC    1
1  pbmc1_Celseq2_1_ACAGGA    2
2  pbmc1_Celseq2_1_ACGTTG    3
3  pbmc1_Celseq2_1_AGACCA    4
4  pbmc1_Celseq2_1_CAACTC    5
(44433, 2)
   gene_idx  cell_idx  expr
0         3         1     1
1         8         1     1
2        17         1     2
3        22         1     1
4        23         1     3
(36234982, 3)
                   gene_ids
0    ENSG00000000003_TSPAN6
1      ENSG00000000005_TNMD
2      ENSG00000000419_DPM1
3     ENSG00000000457_SCYL3
4  ENSG00000000460_C1orf112
(33694, 1)
                 Name Experiment      Method  CBC          CellType
0  pbmc1_SM2_Cell_108      pbmc1  Smart-seq2  NaN  Cytotoxic T cell
1  pbmc1_SM2_Cell_115      pbmc1  Smart-seq2  NaN  Cytotoxic T cell
2  pbmc1_SM2_Cell_133      pbmc1  Smart-seq2  NaN  Cytotoxic T cell
3  pbmc1_SM2_Cell_142      pbmc1  Smart-seq2  NaN  Cytotoxic T cell
4  pbmc1_SM2_Cell_143      pbmc1  Smart-seq2  NaN  Cytotoxic T cell
(31021, 5)


## Filter Data by Exprimental Details

We will only use the 10x data from experiment pbmc1

In [8]:
# first lets take a look at the data
uniq_methods = meta_info["Method"].unique()
print(f"Methods used: {uniq_methods}")

uniq_expr = meta_info["Experiment"].unique()
print(f"Experiment IDs: {uniq_expr}")

# now lets get the names of the cells of interest
names_keep = meta_info[(meta_info["Method"].isin(method_keep)) &
                       (meta_info["Experiment"].isin(experiment_keep))]
names_keep = names_keep["Name"]

if names_keep.shape[0] != num_cells_expected:
    assert False, "Incorrect number of cells from names_keep."

# now get the indices of the cells
cells_keep = cell_info[cell_info["cell_names"].isin(names_keep)]
if cells_keep.shape[0] != num_cells_expected:
    assert False, "Incorrect number of cells from cells_keep."

# using this index, get the count matrix using only the cells of interest
pbmc1_a_mm = count_matr[count_matr["cell_idx"].isin(cells_keep["idx"])]

# now format it to the dense repr
pbmc1_a_dense = pd.crosstab(index=pbmc1_a_mm["gene_idx"],
                            columns=pbmc1_a_mm["cell_idx"],
                            values=pbmc1_a_mm["expr"],
                            aggfunc=sum,
                            dropna=True)
pbmc1_a_dense = pbmc1_a_dense.fillna(0)
if pbmc1_a_dense.shape[1] != num_cells_expected:
    assert False, "Incorrect number of cells after reshaping."
print(f"Filtered table size: {pbmc1_a_dense.shape}")

Methods used: ['Smart-seq2' 'CEL-Seq2' '10x Chromium V2 A' '10x Chromium V2 B'
 '10x Chromium V3' 'Drop-seq' 'Seq-Well' 'inDrops' '10x Chromium V2']
Experiment IDs: ['pbmc1' 'pbmc2']
Filtered table size: (20430, 3222)


## Filter Data by Expression value

In [9]:
pbmc1_a_expr = pbmc1_a_dense[pbmc1_a_dense.sum(axis=1) > min_num_cells]
print(f"Filtered table size: {pbmc1_a_expr.shape}")

# the pbmc1_a_expr is 1-indexed
gene_pass_idx = pbmc1_a_expr.index.to_numpy()-1

gene_pass = gene_info.iloc[gene_pass_idx]


Filtered table size: (20430, 3222)


## Join with metadata

In [10]:
cell_meta_info = cell_info.merge(meta_info, left_on=["cell_names"], right_on=["Name"])

# first transpose
pbmc1_a_df = pbmc1_a_expr.transpose()
pbmc1_a_df.columns = gene_pass["gene_ids"]
expr_col = pbmc1_a_df.columns

# now merge
pbmc1_a_df = cell_meta_info.merge(pbmc1_a_df, left_on=["idx"], right_on=["cell_idx"])
col_interest = ["CellType"]
col_interest = col_interest + expr_col.tolist()
pbmc1_a_df = pbmc1_a_df[col_interest]

# fiter out specific cell types if needed
# if we are only using the cell types shared across all data types
sm2_cell_types = ['CD14+ monocyte', 'Cytotoxic T cell',
                  'CD16+ monocyte', 'B cell',
                  'CD4+ T cell', 'Megakaryocyte']
if out_file_id == "pbmc_rep1_10xV2a_sm2_cells":
  pbmc1_a_df = pbmc1_a_df[pbmc1_a_df['CellType'].isin(sm2_cell_types)]


# now we transpose
pbmc1_a_df = pbmc1_a_df.transpose()

In [11]:
pbmc1_a_df

,0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3131,3132,3133,3134,3135,3136,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3155,3156,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221
CellType,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,...,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD14+ monocyte,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell
ENSG00000000003_TSPAN6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
ENSG00000000419_DPM1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,1,0,0,0,1,1,0,1
ENSG00000000457_SCYL3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
ENSG00000000460_C1orf112,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283073_RP11-834C11.15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000283078_RP11-11M20.4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ENSG00000283103_LLNLR-245B6.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
ENSG00000283117_CTD-2060L22.1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Read in the mixture file

In [12]:

def read_files(data_path, file_idx, file_name):
  pbmc_rep1_pseudobulk_file = os.path.join(data_path, f"{file_name}_pseudo_{file_idx}.pkl")
  pbmc_rep1_prop_file = os.path.join(data_path, f"{file_name}_prop_{file_idx}.pkl")
  pbmc_rep1_gene_file = os.path.join(data_path, f"{file_name}_genes.pkl")

  pseudobulk_path = Path(pbmc_rep1_pseudobulk_file)
  prop_path = Path(pbmc_rep1_prop_file)
  gene_path = Path(pbmc_rep1_gene_file)

  prop_df = pickle.load( open( prop_path, "rb" ) )
  pseudobulks_df = pickle.load( open( pseudobulk_path, "rb" ) )
  gene_df = pickle.load( open( gene_path, "rb" ) )

  return (pseudobulks_df, prop_df, gene_df)

# get all the augmented data
X_train, Y_train, gene_df = read_files(aug_data_path, 0, out_file_id)

X_train.columns = gene_df["gene_ids"]

# now we transpose
X_train = X_train.transpose()
X_train.columns = range(X_train.shape[1])

In [13]:
Y_train

,CD14+ monocyte,Cytotoxic T cell,CD16+ monocyte,B cell,CD4+ T cell,Megakaryocyte
0,0.455,0.0075,0.217,0,0.066,0.2545
0,0.0105,0.727,0.0525,0.076,0.0405,0.0935
0,0.177,0.1255,0.0025,0.0435,0.2505,0.401
0,0.03,0.0135,0,0.491,0.2845,0.181
0,0.6495,0,0.03,0.0015,0.309,0.01
...,...,...,...,...,...,...
0,0.002,0.7515,0.1305,0.062,0.0355,0.0185
0,0.002,0.007,0.1285,0.2065,0.6085,0.0475
0,0.0605,0.0405,0.2725,0.023,0.5595,0.044
0,0.2605,0.004,0.113,0.0195,0.59,0.013


# Write out Data

In [14]:
# write out the scRNA-seq signature matrix
sig_out_file = os.path.join(cybersort_path, f"{out_file_id}_0_cybersort_sig.tsv.gz")
sig_out_path = Path(sig_out_file)

pbmc1_a_df.to_csv(sig_out_path, sep='\t',header=False)

# write out the bulk RNA-seq mixture matrix
sig_out_file = os.path.join(cybersort_path, f"{out_file_id}_0_cybersort_mix.tsv.gz")
sig_out_path = Path(sig_out_file)

X_train.to_csv(sig_out_path, sep='\t')
